
 
<h1 style="text-align: center;">FINE TUNING LAG-LLAMA MODEL</h1>


## **Part 1:** Set up

### Part 1.1 Imports

In [1]:
#IMPORTS 
## Libraries
import pickle
import os
import time 
import torch


# Import custom functionalities 
from data_prep import load_and_preprocess_data, general_revenue_dataframe, split_univariate_data, get_lag_llama_dataset
from config import MODEL_PATH, SEED, seed_everything, lag_llama_environment, find_device
from models import lag_llama_estiamtor

/Users/nairabarseghyan/anaconda3/lib/python3.11/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


Using MPS device


### Part 1.2 Install and set up lag-llama environment

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True' # Necessary on MacOS for certain operations
seed_everything(SEED)
lag_llama_environment()

Failed to clone the repository.
fatal: destination path 'lag-llama' already exists and is not an empty directory.



In [3]:
%cd lag-llama

/Users/nairabarseghyan/Desktop/project_name/src/lag-llama


### Part 1.2 import lag-llama libraries

In [4]:

from gluonts.evaluation import Evaluator, make_evaluation_predictions


from lag_llama.gluon.estimator import LagLlamaEstimator

## **Part 2:**  Data

In [5]:
#loading data and adding necessary features
initial_data = load_and_preprocess_data()
univariate_data = general_revenue_dataframe(initial_data)

train_data_raw, test_data = split_univariate_data(univariate_data)
train_data = get_lag_llama_dataset(train_data_raw,frequency = "1D")

# **Part 3:** Fine Tuning

### Part 3.1 Parameters

In [6]:
prediction_length = len(test_data)  # prediction length
prediction_length_train = len(train_data_raw)
num_samples = 1000 # sampled from the distribution for each timestep
llama_device = torch.device(find_device())


Using MPS device


### Part 3.2 Set up estimator

In [7]:
estimator = lag_llama_estiamtor(llama_device, prediction_length, num_samples, LagLlamaEstimator)

### Part 3.2 TRAINING

In [8]:
start_time = time.time()

predictor = estimator.train(
    train_data,
    cache_data=True,
    shuffle_buffer_length=1000,
)
print("Fine - Tuning process of Lag-Llama model is completed in --- %s seconds ---" % (time.time() - start_time))

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/nairabarseghyan/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name          | Type               | Params
-----------------------------------------------------
0 | model         | LagLlamaModel      | 2.4 M 
1 | augmentations | ApplyAugmentations | 0     
-----------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.797     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 17.74135 (best 17.74135), saving model to '/Users/nairabarseghyan/Desktop/project_name/src/lag-llama/lightning_logs/version_0/checkpoints/epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 17.22639 (best 17.22639), saving model to '/Users/nairabarseghyan/Desktop/project_name/src/lag-llama/lightning_logs/version_0/checkpoints/epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 16.81999 (best 16.81999), saving model to '/Users/nairabarseghyan/Desktop/project_name/src/lag-llama/lightning_logs/version_0/checkpoints/epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' reached 16.46832 (best 16.46832), saving model to '/Users/nairabarseghyan/Desktop/project_name/src/lag-llama/lightning_logs/version_0/checkpoints/epoch=3-step=200.ckpt' as top 1
Epoch 4, global step 250: 'train_loss' reached 16.22957 (best 16.22957), saving model to '/Users/nairabarseghyan/Desktop/project_name/

Fine - Tuning process of Lag-Llama model is completed in --- 18006.545348882675 seconds ---


# **Part 4:** Saving the model

In [9]:
lag_llama_model_filename = os.path.join(MODEL_PATH, 'fine_tuned_lag_llama_model.pth')

torch.save(predictor, lag_llama_model_filename)



print("The Fine - Tuning process of Lag-Llama model is completed.")

The Fine - Tuning process of Lag-Llama model is completed.


# **End**